In [1]:
%cd '/app'

/app


In [2]:
%load_ext autoreload
%autoreload 2

In [7]:
import torch
import os
import sys 
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
sys.path.append('/app/src')

In [33]:
load('src/dataset/symbolic_utils.sage')

In [10]:
from loader.data import _load_data

In [11]:
def sequence_to_poly(seq, ring):
    monoms = seq.split('+')
    d = {}
    for monom in monoms:
       m = monom.split()
       if '/' in m[0]:
         a, slash, b = m[:3]
         assert (slash == '/')
         coeff = f'{a[1:]}/{b[1:]}'
       else:
         coeff = m[0][1:]   
         ex = m[1:]
         d[tuple([int(ei[1:]) for ei in ex])] = float(coeff)
      
    return ring(d)
   

In [12]:
task = 'shape'

n, field = 2, 'GF7'
data_name = f'{task}_n={n}_field={field}'
data_path = f'data/{task}/{data_name}/data_{field}_n={n}.test.lex.infix'

dataset = _load_data(data_path)

In [14]:
dataset[0]

{'input': 'C2 E3 E0 + C4 E2 E4 + C5 E2 E3 + C1 E2 E2 + C4 E2 E1 + C2 E2 E0 + C1 E0 E5 + C6 E0 E4 + C2 E0 E3 + C2 E0 E1 + C6 E0 E0 [SEP] C5 E3 E2 + C2 E3 E0 + C3 E2 E6 + C2 E2 E5 + C3 E2 E4 + C1 E2 E3 + C6 E2 E2 + C4 E2 E1 + C2 E2 E0 + C1 E1 E0 + C6 E0 E7 + C1 E0 E6 + C6 E0 E5 + C1 E0 E4 + C6 E0 E3 + C5 E0 E2 + C4 E0 E1',
 'target': 'C1 E1 E0 + C2 E0 E4 + C6 E0 E3 + C4 E0 E2 + C2 E0 E1 + C1 E0 E0 [SEP] C1 E0 E5 + C6 E0 E4 + C2 E0 E3 + C2 E0 E1 + C6 E0 E0',
 'input_mask': None,
 'target_mask': None}

In [20]:
p

'C5 E3 E2 + C2 E3 E0 + C3 E2 E6 + C2 E2 E5 + C3 E2 E4 + C1 E2 E3 + C6 E2 E2 + C4 E2 E1 + C2 E2 E0 + C1 E1 E0 + C6 E0 E7 + C1 E0 E6 + C6 E0 E5 + C1 E0 E4 + C6 E0 E3 + C5 E0 E2 + C4 E0 E1'

In [26]:
def get_raw_dataset(dataset, ring):
    raw_dataset = []
    for sample in dataset:
        Fstr, Gstr = sample['input'], sample['target']
        F, G = [], []
        
        for fstr in Fstr.split(' [SEP] '):
            f = sequence_to_poly(fstr, ring)
            F.append(f)
        
        for gstr in Gstr.split(' [SEP] '):
            g = sequence_to_poly(gstr, ring)
            G.append(g)
            
        raw_dataset.append((F, G))

    return raw_dataset

In [27]:
ring = PolynomialRing(GF(7), 'x', n, order='lex')
raw_dataset = get_raw_dataset(dataset, ring)

In [30]:
raw_dataset[0]

([2*x0^3 - 3*x0^2*x1^4 - 2*x0^2*x1^3 + x0^2*x1^2 - 3*x0^2*x1 + 2*x0^2 + x1^5 - x1^4 + 2*x1^3 + 2*x1 - 1,
  -2*x0^3*x1^2 + 2*x0^3 + 3*x0^2*x1^6 + 2*x0^2*x1^5 + 3*x0^2*x1^4 + x0^2*x1^3 - x0^2*x1^2 - 3*x0^2*x1 + 2*x0^2 + x0 - x1^7 + x1^6 - x1^5 + x1^4 - x1^3 - 2*x1^2 - 3*x1],
 [x0 + 2*x1^4 - x1^3 - 3*x1^2 + 2*x1 + 1, x1^5 - x1^4 + 2*x1^3 + 2*x1 - 1])

In [21]:
ring = PolynomialRing(GF(7), 'x', n, order='lex')

p = dataset[0]['input'].split(' [SEP] ')[-1]

sequence_to_poly(p, ring)

-2*x0^3*x1^2 + 2*x0^3 + 3*x0^2*x1^6 + 2*x0^2*x1^5 + 3*x0^2*x1^4 + x0^2*x1^3 - x0^2*x1^2 - 3*x0^2*x1 + 2*x0^2 + x0 - x1^7 + x1^6 - x1^5 + x1^4 - x1^3 - 2*x1^2 - 3*x1

In [9]:
task = 'shape'
encoding = 'standard'

n, field = 2, 'GF7'
data_name = f'{task}_n={n}_field={field}_init'
data_path = f'data/{task}/{data_name}/data_{field}_n={n}.test.lex.infix'

data_config_path = f'config/{data_name}.yaml'
_save_path = f'{field}_n={n}_ep=8_bs=16'
save_path = f'results/{task}/{encoding}_embedding_init/{_save_path}'
# save_path = f'results/{task}/{encoding}/dryrun'

In [10]:
bag = load_pretrained_bag(save_path)
config, model, tokenizer = bag['config'], bag['model'], bag['tokenizer']

In [11]:
from loader.data import _load_data
from loader.data import SimpleDataCollator
from torch.utils.data import DataLoader
import numpy as np 
from tqdm import tqdm

batch_size = 500
dataloader = data_path
disable_tqdm = False
max_length = 10000

if isinstance(dataloader, str):
    dataset = _load_data(dataloader)
    dc = SimpleDataCollator(tokenizer)
    dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=dc, shuffle=False)

# load model    
if isinstance(model, str):
    bag = load_pretrained_bag(model)
    config, model, tokenizer = bag['config'], bag['model'], bag['tokenizer']
else:
    assert(tokenizer is not None)


model.cuda()

with torch.no_grad():
    hits = []
    dataloader = tqdm(dataloader, disable=disable_tqdm)  if not disable_tqdm else dataloader
    for batch in dataloader:
        max_length = min(max_length, batch['labels'].shape[1] + 1)
        outputs = model.greedy_generate(batch['encoder_input'].cuda(), max_length=max_length, encoder_padding_mask=batch['encoder_padding_mask'].cuda())
        pred = tokenizer.batch_decode(outputs.cpu().numpy(), skip_special_tokens=True)
        target = tokenizer.batch_decode(batch['labels'], skip_special_tokens=True)
        
        hits += [p == t for p, t in zip(pred, target)]
        
    ret = {'acc': np.array(hits, dtype=float).mean(), 'hits': hits}

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:04<00:00,  2.36s/it]


In [12]:
ret['acc']

0.579

In [132]:
ret['hits'][:10]

[True, False, True, True, True, False, True, False, True, False]

In [133]:
batch.keys()

dict_keys(['encoder_input', 'decoder_input', 'encoder_padding_mask', 'decoder_padding_mask', 'labels'])

In [134]:
batch  = next(iter(dataloader))

In [28]:
%%time
outputs = model.greedy_generate(batch['encoder_input'].cuda(), 
                                encoder_attention_mask=None,
                                encoder_padding_mask=batch['encoder_padding_mask'].cuda(),
                                max_length=max_length)

CPU times: user 22.6 s, sys: 17.1 ms, total: 22.6 s
Wall time: 1.98 s


In [150]:
outputs[:4]

tensor([[36,  4, 12, 11, 32,  4, 11, 11, 40,  4, 11, 12, 38, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36],
        [36,  4, 12, 11, 32,  4, 11, 13, 40,  4, 11, 14, 32,  5, 11, 13, 32,  5,
         11, 12, 32,  5, 11, 11, 38, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36],
        [36,  4, 12, 11, 32,  5, 11, 11, 40,  4, 11, 15, 38, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36],
        [36,  4, 12, 11, 32,  8, 11, 14, 32,  9, 11, 12, 32,  9, 11, 11, 40,  4,
         11, 15, 32,  9, 11, 14, 32,  8, 11, 13, 32,  8, 11, 12, 38, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36]], device='cuda:0')

In [142]:
batch['labels'][:4]

tensor([[ 4, 12, 11, 32,  4, 11, 11, 40,  4, 11, 12, 38, 36, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36],
        [ 4, 12, 11, 32,  4, 11, 13, 40,  4, 11, 14, 32,  8, 11, 11, 38, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36],
        [ 4, 12, 11, 32,  5, 11, 11, 40,  4, 11, 15, 38, 36, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36],
        [ 4, 12, 11, 32,  8, 11, 14, 32,  9, 11, 12, 32,  9, 11, 11, 40,  4, 11,
         15, 32,  9, 11, 14, 32,  8, 11, 13, 32,  8, 11, 12, 38, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36]])

In [151]:
tokenizer.batch_decode(outputs.cpu().numpy(), skip_special_tokens=True)[1]

'C1 E1 E0 + C1 E0 E2 [SEP] C1 E0 E3 + C2 E0 E2 + C2 E0 E1 + C2 E0 E0'

In [147]:
tokenizer.batch_decode(batch['labels'], skip_special_tokens=True)[1]

'C1 E1 E0 + C1 E0 E2 [SEP] C1 E0 E3 + C5 E0 E0'

In [148]:
tokenizer.batch_decode(batch['encoder_input'], skip_special_tokens=True)[1]


'C1 E1 E0 + C1 E0 E2 [SEP] C1 E3 E1 + C1 E2 E3 + C5 E2 E2 + C5 E1 E4 + C1 E1 E2 + C5 E1 E1 + C3 E1 E0 + C6 E0 E3 + C3 E0 E2 + C2 E0 E1 + C5 E0 E0 [SEP] C4 E5 E2 + C4 E4 E4 + C4 E4 E1 + C4 E3 E3 + C6 E3 E2 + C5 E3 E1 + C3 E2 E5 + C3 E2 E4 + C5 E2 E3 + C3 E2 E1 + C1 E2 E0 + C2 E1 E4 + C1 E1 E3 + C1 E1 E2 + C2 E0 E4 + C5 E0 E3 + C1 E0 E2 + C3 E0 E1 + C2 E0 E0'

In [71]:
tokenizer.vocab

{'E9': 20,
 'C0': 3,
 'E12': 23,
 '[UNK]': 41,
 '/': 35,
 'E10': 21,
 'E2': 13,
 'C2': 5,
 'E17': 28,
 '[SEP]': 40,
 '^': 34,
 'E14': 25,
 'E18': 29,
 '</s>': 38,
 '<s>': 37,
 'E3': 14,
 '+': 32,
 'C3': 6,
 'E8': 19,
 'E13': 24,
 'E0': 11,
 'E5': 16,
 '[PAD]': 36,
 'E4': 15,
 'C4': 7,
 'E20': 31,
 'C1': 4,
 'C5': 8,
 '[C]': 2,
 'C6': 9,
 '[E]': 10,
 'E19': 30,
 '[CLS]': 39,
 '*': 33,
 'E16': 27,
 'E1': 12,
 'E11': 22,
 'x0': 0,
 'x1': 1,
 'E15': 26,
 'E7': 18,
 'E6': 17}